# Deliverable 2. Create a Customer Travel Destinations Map

----

1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Geoapify API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found".

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame. In the point for each city add:

    * The city name
    
    * The country code
    
    * The weather description
    
    * The point's size should be the maximum temperature for the city

15. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.

---

## Ensure that the dependencies and the Geoapify API key is imported correctly

In [27]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`

In [28]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Arraial Do Cabo,BR,-22.9661,-42.0278,73.78,78,100,18.21,overcast clouds
1,1,Tuktoyaktuk,CA,69.4541,-133.0374,3.20,78,40,20.71,scattered clouds
2,2,Whitianga,NZ,-36.8333,175.7000,63.64,95,100,17.98,light rain
3,3,Souillac,MU,-20.5167,57.5167,75.58,78,40,9.22,scattered clouds
4,4,Barrow,US,71.2906,-156.7887,10.42,67,100,14.97,overcast clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation

In [29]:
# Prompt the user to enter minimum and maximum temperature criteria.
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature that you would like for your trip? "))
max_temp = float(input("What is the maximum temperature that you would like for your trip? "))

## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`

In [30]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method. Between 70 and 95
preferred_cities_df = city_data_df.loc[
    (city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)
]

# Display sample data
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Arraial Do Cabo,BR,-22.9661,-42.0278,73.78,78,100,18.21,overcast clouds
3,3,Souillac,MU,-20.5167,57.5167,75.58,78,40,9.22,scattered clouds
6,6,Barpali,IN,21.2000,83.5833,70.61,49,14,2.86,few clouds
13,13,Rikitea,PF,-23.1203,-134.9692,74.88,74,3,17.36,clear sky
15,15,Sainte-Rose,GP,16.3324,-61.6979,75.24,83,75,0.00,broken clouds
21,21,Pirapora,BR,-17.3450,-44.9419,71.20,93,100,3.40,overcast clouds
22,22,Vila Velha,BR,-20.3297,-40.2925,75.15,95,100,17.27,overcast clouds
23,23,Saint-Leu,RE,-21.1500,55.2833,75.31,73,14,3.49,few clouds
25,25,Beberibe,BR,-4.1797,-38.1306,77.54,81,100,8.77,overcast clouds
26,26,Vaini,TO,-21.2000,-175.2000,80.76,94,75,16.11,light rain


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows

In [31]:
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [32]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities_df = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Arraial Do Cabo,BR,-22.9661,-42.0278,73.78,78,100,18.21,overcast clouds
3,3,Souillac,MU,-20.5167,57.5167,75.58,78,40,9.22,scattered clouds
6,6,Barpali,IN,21.2000,83.5833,70.61,49,14,2.86,few clouds
13,13,Rikitea,PF,-23.1203,-134.9692,74.88,74,3,17.36,clear sky
15,15,Sainte-Rose,GP,16.3324,-61.6979,75.24,83,75,0.00,broken clouds
...,...,...,...,...,...,...,...,...,...,...
641,641,Vallenar,CL,-28.5708,-70.7581,71.94,44,0,3.76,clear sky
643,643,Ternate,ID,0.8000,127.4000,83.21,73,80,5.77,broken clouds
653,653,Acapulco,MX,16.8634,-99.8901,76.10,81,50,6.96,scattered clouds
655,655,Puerto Escondido,MX,15.8500,-97.0667,76.96,81,35,3.42,scattered clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"

In [33]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame

In [34]:
# Create a new column "Hotel Name".
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Arraial Do Cabo,BR,73.78,overcast clouds,-22.9661,-42.0278,
3,Souillac,MU,75.58,scattered clouds,-20.5167,57.5167,
6,Barpali,IN,70.61,few clouds,21.2000,83.5833,
13,Rikitea,PF,74.88,clear sky,-23.1203,-134.9692,
15,Sainte-Rose,GP,75.24,broken clouds,16.3324,-61.6979,
21,Pirapora,BR,71.20,overcast clouds,-17.3450,-44.9419,
22,Vila Velha,BR,75.15,overcast clouds,-20.3297,-40.2925,
23,Saint-Leu,RE,75.31,few clouds,-21.1500,55.2833,
25,Beberibe,BR,77.54,overcast clouds,-4.1797,-38.1306,
26,Vaini,TO,80.76,light rain,-21.2000,-175.2000,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city

In [35]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

## Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found"

In [36]:
# Iterate through the hotel DataFrame

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set up the base URL for the Geoapify Places API.
    base_url = "https://api.geoapify.com/v2/places"

    # Make request and retrieve the JSON data by using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found set the hotel name as "No hotel found".
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Arraial Do Cabo - nearest hotel: No hotel found
Souillac - nearest hotel: Hotel SkylineInn
Barpali - nearest hotel: No hotel found
Rikitea - nearest hotel: Chez Bianca & Benoit
Sainte-Rose - nearest hotel: No hotel found
Pirapora - nearest hotel: Hotel Mundial
Vila Velha - nearest hotel: Hotel Prainha
Saint-Leu - nearest hotel: Iloha Seaview Hôtel
Beberibe - nearest hotel: Pousada Ibitu
Vaini - nearest hotel: Keleti International Resort
Butaritari - nearest hotel: No hotel found
Bandarbeyla - nearest hotel: No hotel found
Angoche - nearest hotel: Hotel Quirimbas
Alofi - nearest hotel: No hotel found
West Melbourne - nearest hotel: Courtyard by Marriott - Melbourne West
Bubaque - nearest hotel: Chez Julio
Kapaa - nearest hotel: Pono Kai Resort
Yulara - nearest hotel: Outback Pioneer Hotel & Lodge
Labuhan - nearest hotel: No hotel found
Saint-Philippe - nearest hotel: Le Baril
Vao - nearest hotel: Nataiwatch
Busselton - nearest hotel: No hotel found
Lazaro Cardenas 

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Arraial Do Cabo,BR,73.78,overcast clouds,-22.9661,-42.0278,No hotel found
3,Souillac,MU,75.58,scattered clouds,-20.5167,57.5167,Hotel SkylineInn
6,Barpali,IN,70.61,few clouds,21.2000,83.5833,No hotel found
13,Rikitea,PF,74.88,clear sky,-23.1203,-134.9692,Chez Bianca & Benoit
15,Sainte-Rose,GP,75.24,broken clouds,16.3324,-61.6979,No hotel found
...,...,...,...,...,...,...,...
641,Vallenar,CL,71.94,clear sky,-28.5708,-70.7581,Hotel del Marqués
643,Ternate,ID,83.21,broken clouds,0.8000,127.4000,Boulivard ternate
653,Acapulco,MX,76.10,scattered clouds,16.8634,-99.8901,Hotel del Valle
655,Puerto Escondido,MX,76.96,scattered clouds,15.8500,-97.0667,Hotel Barlovento


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`

In [45]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

# Display sample data
clean_hotel_df.head(10) 

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Arraial Do Cabo,BR,73.78,overcast clouds,-22.9661,-42.0278,No hotel found
3,Souillac,MU,75.58,scattered clouds,-20.5167,57.5167,Hotel SkylineInn
6,Barpali,IN,70.61,few clouds,21.2000,83.5833,No hotel found
13,Rikitea,PF,74.88,clear sky,-23.1203,-134.9692,Chez Bianca & Benoit
15,Sainte-Rose,GP,75.24,broken clouds,16.3324,-61.6979,No hotel found
21,Pirapora,BR,71.20,overcast clouds,-17.3450,-44.9419,Hotel Mundial
22,Vila Velha,BR,75.15,overcast clouds,-20.3297,-40.2925,Hotel Prainha
23,Saint-Leu,RE,75.31,few clouds,-21.1500,55.2833,Iloha Seaview Hôtel
25,Beberibe,BR,77.54,overcast clouds,-4.1797,-38.1306,Pousada Ibitu
26,Vaini,TO,80.76,light rain,-21.2000,-175.2000,Keleti International Resort


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the `Vacation_Search`` folder

In [46]:
# Set the output file name
output_data_file = 'WeatherPy_vacation.csv'

# Export the city_data DataFrame into a CSV file
clean_hotel_df.to_csv(output_data_file, index_label='City_ID')

## Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame.

In the point for each city add:

* The city name
* The country code
* The weather description
* The point's size should be the maximum temperature for the city

In [50]:
# Configure the map plot
clean_hotel_map = clean_hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country", "Current Description"]
)

# Display the map
clean_hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Current Description)